In [ ]:
import os 
import numpy as np 
import matplotlib.pyplot as plt 
import torch 
from data import cfg
from layers.functions.prior_box import PriorBox
from utils.nms.py_cpu_nms import py_cpu_nms
import cv2
from models.faceboxes import FaceBoxes
from utils.box_utils import decode
from utils.timer import Timer
from utils.pred_utils import get_pred, plot_result

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

classes = {'background': 0, 'face': 1, 'fask_mask': 2}

num_classes = len(classes) # inclcude background 
resize = 1 

model = FaceBoxes(phase='test', size=None, num_classes=len(classes), pretrained=False)
cp_f = 'model_checkpoints/checkpoint_FaceBoxes.pth' 
checkpoint = torch.load(cp_f)
model.load_state_dict(checkpoint)
model.eval() 
print('Finished loading model...')
model = model.to(device)

# adjustable threshold 
confidence_threshold = 0.15
nms_threshold = 0.15
vis_thres = 0.5 
num_classes = 3 # inclcude background 
resize = 1 

In [ ]:
# test image 
image_path = 'data/FaceMask/val/9_Press_Conference_Press_Conference_9_924.jpg'
img_raw = cv2.imread(image_path, cv2.IMREAD_COLOR)
img = np.float32(img_raw)
im_height, im_width, _ = img.shape
scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
img -= (104, 117, 123)
img = img.transpose(2, 0, 1)
img = torch.from_numpy(img).unsqueeze(0)
img = img.to(device)
scale = scale.to(device)
loc, conf = model(img) 
priorbox = PriorBox(cfg, image_size=(im_height, im_width))
priors = priorbox.forward()
priors = priors.to(device)
prior_data = priors.data
# convert back to pixel 
boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])
boxes = boxes * scale / resize
boxes = boxes.cpu().numpy()

dets = get_pred(boxes, conf, num_classes, confidence_threshold, nms_threshold)

# plot result using cv2 
plot_result(img_raw, dets) 